# Fase 1: Extracción de Datos de API de MoviesDataset

In [84]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select


LINK API: https://rapidapi.com/SAdrian/api/moviesdatabase

- Películas desde el 2010 hasta la actualidad.

- Películas que pertenezcan a los siguientes géneros: Drama, Comedy, Action.

- NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

In [85]:
#creamos las variables
dic_API_Pelis = {'Tipo':[],
                 'Nombre':[],
                 'Año_estreno':[],
                 'Mes_estreno': [],
                 'Id':[]}

#creamos las variables relacionadas con el ENDPOINT
url = "https://moviesdatabase.p.rapidapi.com/titles"

API_key = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}
page = 1
querystring = {"startYear":"2010","endYear":"2024","genre":"Drama", "genre":"Comedy", "genre":"Action", "limit":"50", "page":page} # limite 50 



### Vamos a investigar primero en una película

In [86]:
#realizamos la llamada
response = requests.get(url, headers=headers, params=querystring)
response.status_code


200

In [87]:
#convertimos la respuesta a json formato
js_response = response.json()
js_response.keys()

dict_keys(['page', 'next', 'entries', 'results'])

In [88]:
#inspeccionamos cada una de las keys()

In [89]:
#Nos da informacion no necesaria
js_response['page']

'1'

In [90]:
# Nos da el link de la proxima pagina
js_response['next']


'/titles?endYear=2024&genre=Action&page=2&limit=50&startYear=2010'

In [91]:
#Nos da informacion no necesaria
js_response['entries']

50

In [92]:
info = js_response['results']
info

[{'_id': '61e58b91d735dff3f943afb9',
  'id': 'tt10064424',
  'primaryImage': {'id': 'rm479879936',
   'width': 1200,
   'height': 1800,
   'url': 'https://m.media-amazon.com/images/M/MV5BZTIwZmViMWItMDA5Ny00OTczLTg2NzYtYzEwM2ZjOTI3NmVlXkEyXkFqcGdeQXVyMzY0MTE3NzU@._V1_.jpg',
   'caption': {'plainText': 'Vampire: The Masquerade - Bloodlines 2 (2024)',
    '__typename': 'Markdown'},
   '__typename': 'Image'},
  'titleType': {'displayableProperty': {'value': {'plainText': 'Video Game',
     '__typename': 'Markdown'},
    '__typename': 'DisplayableTitleTypeProperty'},
   'text': 'Video Game',
   'id': 'videoGame',
   'isSeries': False,
   'isEpisode': False,
   'categories': [{'value': 'gaming', '__typename': 'TitleTypeCategory'}],
   'canHaveEpisodes': False,
   '__typename': 'TitleType'},
  'titleText': {'text': 'Vampire: The Masquerade - Bloodlines 2',
   '__typename': 'TitleText'},
  'originalTitleText': {'text': 'Vampire: The Masquerade - Bloodlines 2',
   '__typename': 'TitleText'},
 

In [93]:
len(info)

50

### 50 listas = 50 peliculas

Cada lista es un diccionario = diccionario de cada pelicula

Keys de cada diccionario:
['_id', 'id', 'primaryImage', 'titleType', 'titleText', 'originalTitleText', 'releaseYear', 'releaseDate']



In [94]:
#Inspeccionamos una lista:
info[0].keys()

dict_keys(['_id', 'id', 'primaryImage', 'titleType', 'titleText', 'originalTitleText', 'releaseYear', 'releaseDate'])

In [95]:
# Nos da info no necesaria
info[0]['_id']

'61e58b91d735dff3f943afb9'

In [96]:
# Nos da info no necesaria
info[0]['primaryImage']

{'id': 'rm479879936',
 'width': 1200,
 'height': 1800,
 'url': 'https://m.media-amazon.com/images/M/MV5BZTIwZmViMWItMDA5Ny00OTczLTg2NzYtYzEwM2ZjOTI3NmVlXkEyXkFqcGdeQXVyMzY0MTE3NzU@._V1_.jpg',
 'caption': {'plainText': 'Vampire: The Masquerade - Bloodlines 2 (2024)',
  '__typename': 'Markdown'},
 '__typename': 'Image'}

In [97]:
# Nos da info no necesaria
info[0]['originalTitleText']

{'text': 'Vampire: The Masquerade - Bloodlines 2', '__typename': 'TitleText'}

In [98]:
month_peli = info[5]['releaseDate']['month'] # Hay que hacer un try/except ya que hay películas que no tienen esta información
month_peli

10

In [107]:
info[0]['releaseDate']['month']

In [99]:
year_peli = info[0]['releaseYear']['year']
year_peli

2024

In [100]:
nombre_peli = info[0]['titleText']['text']
nombre_peli

'Vampire: The Masquerade - Bloodlines 2'

In [101]:
tipo_peli= info[49]['titleType']['text']
tipo_peli

'Movie'

In [102]:
id_peli = info[49]['id']
id_peli

'tt15009428'

### Vamos a buscar y almacenar todas las paginas (con un límite de 10 páginas)

In [108]:
url = "https://moviesdatabase.p.rapidapi.com/titles"

API_key = '039163dca0msh7ecfddf47a57b42p193f39jsn80c27500a5c2'
headers = {"X-RapidAPI-Key": API_key,"X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"}

page=1

#creamos una lista para almanacer los códigos de respuesta de las llamadas
response_lista = []

while page<=10:
   
    #Definimos los parámetros para la llamada
    querystring = {"startYear":"2010","endYear":"2024","genre":"Drama", "genre":"Comedy", "genre":"Action", "limit":"50", "page":page} # limite 50 
    
    #Realizamos la llamada
    response_pelis = requests.get(url, headers=headers, params=querystring)
    
    #Guardamos el código del resultado de la llamada en una lista
    response_lista.append(response_pelis.status_code)

    #Convertimos la respuesta de la llamada a formato json
    js_response_pelis = response_pelis.json()

    #Sumamos una página
    page+=1


    #guardamos la parte dónde tenemos la infomación en una variable
    info_pelis = js_response_pelis['results']

    #guardamos toda la información en el diccionario con un for loop
    
    for peli in info_pelis:
        
        dic_API_Pelis['Tipo'].append(peli['titleType']['text'])
        dic_API_Pelis['Nombre'].append(peli['titleText']['text'])
        dic_API_Pelis['Id'].append(peli['id'])
        dic_API_Pelis['Año_estreno'].append(peli['releaseYear']['year'])

        #Hacemos un try/expect por si falta alguna información             
        try:
            dic_API_Pelis['Mes_estreno'].append(peli['releaseDate']['month'])
        except:
            
            dic_API_Pelis['Mes_estreno'].append('error')            

    

print('terminé')



terminé


In [109]:
#Comrpobamos cuantos resultados tenemos
print(len(response_lista))

10


In [110]:
#Visualizamos nuestro diccionario
dic_API_Pelis


{'Tipo': ['Video Game',
  'Movie',
  'TV Series',
  'TV Series',
  'Movie',
  'Movie',
  'Movie',
  'TV Series',
  'Movie',
  'TV Series',
  'TV Series',
  'Video Game',
  'Movie',
  'TV Series',
  'TV Series',
  'Movie',
  'Movie',
  'Movie',
  'TV Series',
  'Movie',
  'TV Series',
  'TV Series',
  'TV Series',
  'Video Game',
  'Movie',
  'TV Episode',
  'Movie',
  'TV Series',
  'TV Episode',
  'TV Episode',
  'Movie',
  'TV Series',
  'TV Episode',
  'TV Series',
  'Video Game',
  'Short',
  'Video Game',
  'Movie',
  'Movie',
  'TV Series',
  'TV Series',
  'TV Episode',
  'Movie',
  'Movie',
  'TV Mini Series',
  'Video Game',
  'Movie',
  'TV Series',
  'TV Series',
  'TV Series',
  'Movie',
  'TV Series',
  'TV Episode',
  'TV Series',
  'TV Series',
  'Movie',
  'TV Series',
  'Video Game',
  'TV Episode',
  'Video Game',
  'Movie',
  'TV Episode',
  'Movie',
  'Movie',
  'TV Series',
  'TV Episode',
  'TV Episode',
  'TV Episode',
  'TV Episode',
  'TV Episode',
  'TV Episod

In [111]:
#Almacenamos la infomacion en distintas listas, cada una corresponde a una key del diccionario
lista_tipos = dic_API_Pelis['Tipo']
lista_nombre = dic_API_Pelis['Nombre']
lista_anno = dic_API_Pelis['Año_estreno']
lista_mes = dic_API_Pelis['Mes_estreno']
lista_id = dic_API_Pelis['Id']

In [112]:
#Creamos un zip con las listas que hemos creado para que nos guarde la infoamción de cada una de las pelis en una tupla
zip_pelis = zip(lista_tipos, lista_nombre, lista_anno, lista_mes, lista_id)
lista_tuplas_pelis = list(zip_pelis)
lista_tuplas_pelis

[('Video Game',
  'Vampire: The Masquerade - Bloodlines 2',
  2024,
  None,
  'tt10064424'),
 ('Movie', 'Multiverso', 2024, 6, 'tt10225380'),
 ('TV Series', 'Dune: The Prophecy', 2024, None, 'tt10466872'),
 ('TV Series', 'Those About to Die', 2024, 7, 'tt10569934'),
 ('Movie', 'Detour 95', 2024, 10, 'tt10844184'),
 ('Movie', 'The Legend of Johnny Jones', 2024, 10, 'tt10928904'),
 ('Movie', 'Slasher House 3', 2024, 3, 'tt11078340'),
 ('TV Series', 'Bandidos', 2024, 3, 'tt11262142'),
 ('Movie', 'Clank and the Golden Scar', 2024, 7, 'tt11374514'),
 ('TV Series', 'Furies', 2024, 3, 'tt11384300'),
 ('TV Series', 'Cross', 2024, None, 'tt11794812'),
 ('Video Game',
  'Vampire: The Masquerade - Bloodlines 2',
  2024,
  6,
  'tt10064424'),
 ('Movie', 'Multiverso', 2024, None, 'tt10225380'),
 ('TV Series', 'Dune: The Prophecy', 2024, 7, 'tt10466872'),
 ('TV Series', 'Those About to Die', 2024, 10, 'tt10569934'),
 ('Movie', 'Detour 95', 2024, 10, 'tt10844184'),
 ('Movie', 'The Legend of Johnny Jo

In [113]:
#comprobamos si tenemos todas las pelis que queríamos
len(lista_tuplas_pelis)

510

In [116]:
lista_tuplas_pelis[499]

('Short', 'Sekhem', 2024, 3, 'tt28833743')